In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
churndata=spark.read.csv("/home/phani/Desktop/Telecom_Churn_Data_SingTel.csv",header=True,inferSchema=True)

In [3]:
churndata.show(10)


+-----+--------------+---------+------------+------------------+---------------+--------------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+------------------+------------------+---------------------------+-----------------+-----------------+------------------------------+-------+
|State|Account Length|Area Code|Phone Number|International Plan|Voice mail Plan|Num of Voice mail Messages|Total Day Minutes|Total Day Calls|Total day Charge|Total Eve Minutes|Total Eve Calls|Total Eve Charge|Total Night Minutes|Total Night Calls |Total Night Charge|Total International Minutes|Total Intl  Calls|Total Intl Charge|Number Customer Service calls |  Churn|
+-----+--------------+---------+------------+------------------+---------------+--------------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+------------------+----------

In [8]:
churndata.printSchema()

root
 |-- State: string (nullable = true)
 |-- Account Length: integer (nullable = true)
 |-- Area Code: integer (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- International Plan: string (nullable = true)
 |-- Voice mail Plan: string (nullable = true)
 |-- Num of Voice mail Messages: integer (nullable = true)
 |-- Total Day Minutes: double (nullable = true)
 |-- Total Day Calls: integer (nullable = true)
 |-- Total day Charge: double (nullable = true)
 |-- Total Eve Minutes: double (nullable = true)
 |-- Total Eve Calls: integer (nullable = true)
 |-- Total Eve Charge: double (nullable = true)
 |-- Total Night Minutes: double (nullable = true)
 |-- Total Night Calls : integer (nullable = true)
 |-- Total Night Charge: double (nullable = true)
 |-- Total International Minutes: double (nullable = true)
 |-- Total Intl  Calls: integer (nullable = true)
 |-- Total Intl Charge: double (nullable = true)
 |-- Number Customer Service calls : integer (nullable = true)
 |-- C

In [10]:
from pyspark.ml.feature import StringIndexer

In [11]:
label_indexer=StringIndexer(inputCol='Churn',outputCol='label')

In [12]:
intplan_indexer=StringIndexer(inputCol='International Plan',outputCol='int_planindex')
vocplan_indexer=StringIndexer(inputCol='Voice mail Plan',outputCol='voc_planindex')

In [13]:
numericcolumns=["Account Length","Num of Voice mail Messages","Total Day Minutes","Total Day Calls","Total day Charge","Total Eve Minutes","Total Eve Calls","Total Eve Charge","Total Eve Minutes","Total Eve Calls","Total Eve Charge","Total Night Minutes","Total Night Calls ","Total Night Charge","Total International Minutes","Total Intl  Calls","Total Intl Charge","Number Customer Service calls "]

In [14]:
from pyspark.ml.feature import VectorAssembler

In [16]:
assembler=VectorAssembler(inputCols=['int_planindex']+['voc_planindex']+numericcolumns,outputCol='features')

In [18]:
from pyspark.ml import Pipeline

In [19]:
from pyspark.ml.classification import RandomForestClassifier

In [21]:
classifier=RandomForestClassifier(labelCol='label',featuresCol='features')

In [23]:
pipeline=Pipeline(stages=[intplan_indexer,vocplan_indexer,label_indexer,assembler,classifier])

In [24]:
(train,test)=churndata.randomSplit([0.70,0.30])

In [25]:
model=pipeline.fit(train)

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [27]:
predictions=model.transform(train)

In [43]:
evaluator=BinaryClassificationEvaluator()

In [44]:
evaluator.evaluate(predictions,{evaluator.metricName:"areaUnderROC"})

0.8997126694801094

In [45]:
predictions2=model.transform(train)

In [46]:
evaluator.evaluate(predictions2,{evaluator.metricName:"areaUnderROC"})

0.9423938452193018

In [36]:
from pyspark.ml.classification import DecisionTreeClassifier

In [47]:
predictions3=model.transform(test)

In [48]:
evaluator.evaluate(predictions3,{evaluator.metricName:"areaUnderROC"})

0.8997126694801094

In [49]:
predictions4=model.transform(test)

In [50]:
evaluator.evaluate(predictions4,{evaluator.metricName:"areaUnderROC"})

0.8997126694801094